In [1]:
from rl.distribution import Gaussian
from rl.function_approx import FunctionApprox, LinearFunctionApprox
from rl.chapter9.optimal_order_execution import OptimalOrderExecution, PriceAndShares
import numpy as np

from typing import Tuple, Iterator
from rl.markov_decision_process import NonTerminal
from rl.policy import DeterministicPolicy
from rl.approximate_dynamic_programming import ValueFunctionApprox

### Modeling an LPT Price Impact

In [2]:
init_price_mean: float = 100.0
init_price_stdev: float = 0.15
num_shares: int = 100
num_time_steps = 5
beta: float = 0.01
theta: float = 0.01
mu = 0.03
sigma = 0.15
eta_sd = 0.15
rho = 0.80

# price diff : g_{t} = P_t * (beta * R_{t} + theta * X_{t})
price_diff = [
    lambda p_s_x: p_s_x.price * (beta * p_s_x.shares + theta * p_s_x.X)
    for _ in range(num_time_steps)
]

# price dynamics : P_{t + 1} = P_{t} * e^{Z}, where Z ~ N(mu, sigma)
dynamics = [
    lambda p_s_x: Gaussian(mu, sigma).map(lambda x : p_s_x.price * np.exp(x))  # lognormal
    for _ in range(num_time_steps)
]

# feature functions
ffs = [
    lambda p_s_x: p_s_x.state.price,
    lambda p_s_x: p_s_x.state.price * p_s_x.state.shares,
    lambda p_s_x: p_s_x.state.price * p_s_x.state.X,
    lambda p_s_x: p_s_x.state.price * float(p_s_x.state.shares * p_s_x.state.shares),
    lambda p_s_x: p_s_x.state.price * float(p_s_x.state.X * p_s_x.state.X),
    lambda p_s_x: p_s_x.state.price * float(p_s_x.state.X * p_s_x.state.shares),
]

fa: FunctionApprox = LinearFunctionApprox.create(feature_functions=ffs)
init_price_distrib: Gaussian = Gaussian(init_price_mean, init_price_stdev)
init_X_distrib: Gaussian = Gaussian(0.25, eta_sd)


#### Recursive Formulation of Analytic Solution

In [3]:
T = num_time_steps
q = np.exp(mu + sigma ** 2 / 2)
X_init = 0.0

vf_coeff = np.zeros(shape=(6, T))

nu = np.zeros(shape=(3, T))

# order of const, X, R
nu[0, T - 2] = (1 - q) / (2 * beta * (1 + q))
nu[1, T - 2] = (theta * (q * rho - 1)) / (2 * beta * (1 + q))
nu[2, T - 2] = q / (1 + q)

# c^{(0)}_{T - 2}
vf_coeff[0, T - 2] = nu[0, T - 2] * (1 - theta * nu[0, T - 2]) - q * nu[0, T - 2] * (
    1 + theta * nu[0, T - 2]
)
vf_coeff[1, T - 2] = (1 - q) * nu[1, T - 2]
vf_coeff[2, T - 2] = nu[1, T - 2] * (theta * nu[1, T - 2] - beta) + q * nu[1, T - 2] * (
    beta * rho - theta * nu[1, T - 2]
)
vf_coeff[3, T - 2] = -beta * (1 + rho) * nu[2, T - 2]
vf_coeff[4, T - 2] = 2 * nu[2, T - 2]
vf_coeff[5, T - 2] = -theta * nu[2, T - 2]

for k in range(3, T + 1):
    nu[0, T - k] = (q * vf_coeff[4, T - k + 1] - 1) / (
        2 * (q * vf_coeff[5, T - k + 1] - theta)
    )
    nu[1, T - k] = (q * rho * vf_coeff[2, T - k + 1] + beta) / (
        2 * (q * vf_coeff[5, T - k + 1] - theta)
    )
    nu[2, T - k] = (q * vf_coeff[5, T - k + 1]) / (q * vf_coeff[5, T - k + 1] - theta)

    vf_coeff[0, T - k] = (
        nu[0, T - k] * (1 - theta * nu[0, T - k])
        + q * (vf_coeff[0, T - k + 1] + eta_sd ** 2 * vf_coeff[2, T - k])
        - q
        * nu[0, T - k]
        * (vf_coeff[4, T - k + 1] - nu[1, T - k] * vf_coeff[5, T - k + 1])
    )
    vf_coeff[1, T - k] = q * rho * vf_coeff[1, T - k + 1] - nu[1, T - k] * (
        q * vf_coeff[4, T - k + 1] - 1
    )
    vf_coeff[2, T - k] = (
        -nu[1, T - k] * (theta * nu[1, T - k] + beta)
        + rho ** 2 * vf_coeff[2, T - k + 1]
        - q
        * nu[1, T - k]
        * (rho * vf_coeff[3, T - k + 1] - nu[1, T - k] * vf_coeff[5, T - k + 1])
    )
    vf_coeff[3, T - k] = (
        -beta * nu[2, T - k] + q * (1 - nu[2, T - k]) * vf_coeff[4, T - k + 1]
    )
    vf_coeff[4, T - k] = nu[2, T - k] + q * (1 - nu[2, T - k]) * vf_coeff[4, T - k + 1]
    vf_coeff[5, T - k] = -theta * nu[2, T - k]


In [4]:
# optimal strategy from 100 shares at time T - 2 and price = 100.00 is to go 50/50:
nu[0, T - 2] + X_init * nu[1, T - 2] + 100 * nu[2, T - 2]

50.0

In [5]:
n_remaining = 100
opt_sale = []
for k in range(T, 1, -1):
    
    n_sold = nu[0, T - k] + X_init * nu[1, T - k] + n_remaining * nu[2, T - k]
    n_remaining -= n_sold
    opt_sale.append(int(np.round(n_sold)))
opt_sale.append(100 - np.sum(opt_sale))

print(f"optimal sale sequence {opt_sale}")

optimal sale sequence [17, 19, 20, 21, 23]


#### Evaluating the Value function at t = 0

In [6]:
t = 0
P_init = 100
R_init = 100
(P_init * q) * (
    vf_coeff[:, t].T
    @ np.array([1, X_init, X_init**2, X_init * R_init, R_init, R_init **2 ])
)

9089.012490589019

#### Estimating $V^*$ and $\pi^*$ with backwards induction

In [7]:
ooe: OptimalOrderExecution = OptimalOrderExecution(
    shares=num_shares,
    time_steps=num_time_steps,
    avg_exec_price_diff=price_diff,
    price_dynamics=dynamics,
    utility_func=lambda x: x,
    discount_factor=1,
    func_approx=fa,
    initial_price_distribution=init_price_distrib,
    initial_X_distribution=init_X_distrib,
    eta_sd=eta_sd,
    rho=rho,
)


it_vf: Iterator[
    Tuple[ValueFunctionApprox[PriceAndShares], DeterministicPolicy[PriceAndShares, int]]
] = ooe.backward_induction_vf_and_pi()

state: PriceAndShares = PriceAndShares(price=init_price_mean, shares=num_shares)
print("Backward Induction: VF And Policy")
print("---------------------------------")
print()
for t, (vf, pol) in enumerate(it_vf):
    print(f"Time {t:d}")
    print()
    opt_sale: int = pol.action_for(state)
    val: float = vf(NonTerminal(state))
    print(f"Optimal Sales = {opt_sale:d}, Opt Val = {val:.3f}")
    print()
    print("Optimal Weights below:")
    print(vf.weights.weights)
    print()


Backward Induction: VF And Policy
---------------------------------

Time 0

Optimal Sales = 9, Opt Val = 9191.096

Optimal Weights below:
[-6.20878366e+00  6.12929168e-01  6.70240837e+01  3.68268221e-03
 -4.49960504e-03 -6.76952959e-01]

Time 1

Optimal Sales = 18, Opt Val = 8282.856

Optimal Weights below:
[ 0.01559691  1.10587319  0.20269488 -0.00277744  0.20240256 -0.00336973]

Time 2

Optimal Sales = 17, Opt Val = 7124.106

Optimal Weights below:
[ 0.10039902  1.06108909 -0.30916911 -0.00349682  0.23399101  0.00717399]

Time 3

Optimal Sales = 7, Opt Val = 5233.629

Optimal Weights below:
[ 2.54857905e-02  1.02726343e+00  4.93297156e-03 -5.04155387e-03
 -6.77632117e-02 -4.18970429e-04]

Time 4

Optimal Sales = 14, Opt Val = 0.000

Optimal Weights below:
[ 2.10213707e-14  1.00000000e+00 -2.38191635e-14 -1.00000000e-02
 -1.30556023e-13  2.24703696e-15]



The optimal value for Time 0 is not too far off from what we obtained from the analytic formulation!